In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract --quiet
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

--2021-12-20 14:00:03--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75482 (74K) [text/plain]
Saving to: ‘haarcascade_russian_plate_number.xml’

haarcascade_russian 100%[===================>]  73.71K  --.-KB/s    in 0.002s  

2021-12-20 14:00:03 (41.6 MB/s) - ‘haarcascade_russian_plate_number.xml’ saved [75482/75482]



In [2]:
%%writefile app.py
import cv2
from google.colab.patches import cv2_imshow
import pytesseract
import streamlit as st
import numpy as np
from PIL import Image
st.title("NUMBER PLATE SENSING")
img = st.sidebar.file_uploader("Choose an image")
if img is not None:
  img_read = Image.open(img)
  st.image(img,caption='Uploaded Image') 
  if st.button("PREDICT"):
    img = np.array(img_read)
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #gray = cv2.bilateralFilter(gray,13,15,15)
    v = 150.0/img.shape[1]
    dim = (150, int(img.shape[0]*v))
    resize = cv2.resize(img,dim,interpolation = cv2.INTER_AREA)
    number_plate = cv2.CascadeClassifier('/content/haarcascade_russian_plate_number.xml')
    demo_plate = number_plate.detectMultiScale(resize,1.1,10)
    #hImg,wImg,d = img.shape
    #boxes = pytesseract.image_to_boxes(img)
    #for b in boxes.splitlines():
      #b = b.split()
      #x,y,w,h = int(b[1]),int(b[2]),int(b[3]),int(b[4])
      #cv2.rectangle(img,(x,hImg-y),(w,hImg-h),(255,0,0),5)
      #st.image(img,caption="Boxes")
    for (x,y,w,h) in demo_plate:
      cv2.rectangle(resize,(x,y),(x+(3*h),w-x),(255,255,255),4)
      roi = resize[y:y+h,x:x+w] 
      st.title("Required Number")
      #backtorgb = cv2.cvtColor(resize,cv2.COLOR_GRAY2RGB)
      cv2_imshow(resize)
      st.image(resize,caption="Detected number plate")
      op = pytesseract.image_to_string(roi)
      st.write(op)

Overwriting app.py


In [ ]:
!!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)

http://e3e7-35-243-200-149.ngrok.io
